# 実践データ科学入門 2020年度木曜4限

# 第3回 その3 多変数多項式回帰

In [ ]:
%matplotlib inline
#%matplotlib notebook # if necessary to rotate figures in 3D plot
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import art3d
from ipywidgets import interact
from sklearn import linear_model

## 多変数多項式回帰とは

多項式回帰は説明変数 $x$ 1つの場合に $x$, $x^2$, $x^3$ など，$x$ のべき乘の項を回帰変数に用いる手法であった．多変数多項式回帰とは，説明変数が $x_1, x_2, \ldots, x_M$ と複数ある場合にそれらのべき乗の項を回帰変数に用いる手法である．

例えば，2変数，全次数2次であれば

$$
y = a_0 + a_{11} x_1 + a_{12} x_2 + a_{21} (x_1)^2 + a_{22} x_1 x_2 + a_{23} (x_2)^2 + \xi
$$

というモデルを当てはめる方法である．

多項式の最大次数を全次数ではなく，偏次数で考えても別に構わない．2変数，偏次数2次であれば

$$
1, \ x_1, x_1 x_2, \ (x_1)^2, \ (x_1)^2 x_2, \ (x_1)^2 (x_2)^2, \ x_2, \ (x_2)^2, \ x_1 (x_2)^2
$$

を回帰変数として取ることになる．これは回帰モデルの設計をどうするかの問題なので，解析する側が自由に取って構わない．（モデルの妥当性は別に検証する必要がある．）

多変数多項式回帰モデルも，説明変数 $x_j \ (j=1, 2, \ldots, M)$ からなる単項式を回帰変数とする重回帰モデルであると言える．

全次数 $D$ で多変数多項式モデルを作る場合は

$$
y = \sum_{j_1+j_2+\ldots+j_M \le D} a_{j_1, j_2, \ldots, j_M} \, (x_1)^{j_1} (x_2)^{j_2} \ldots (x_M)^{j_M}
$$

とくに2変数の場合は

$$
y = \sum_{j_1+j_2 \le D} a_{j_1, j_2} \, (x_1)^{j_1} (x_2)^{j_2}
= \sum_{d=0}^D \sum_{k=0}^d a_{d, k}(x_1)^k (x_2)^{d-k}
$$

となる．回帰変数の数は

$$
\sum_{d=0}^D \sum_{k=0}^d 1 - 1 = \sum_{d=0}^D (d+1) - 1 = \sum_{d=1}^D d+ D = \frac{D(D+1)}2 + D = \frac{D(D+3)}2 
$$

である．

## では計算してみよう

In [ ]:
# 真のパラメータ
B0 = 1.2
B1 = -1.8
B2 = 2.2
B3 = 1.1
B4 = -1.3
B5 = 0.4

# dataset
N = 1000
X1 = np.random.rand(N) * 3
X2 = np.random.rand(N) * 3
Y = B0 + B1*X1 + B2*X2 + B3*X1**2 + B4*X1*X2 + B5*X2**2 + np.random.randn(X1.size)

In [ ]:
# フィッティングに用いる多項式の最大次数
D = 2

# 回帰変数の数
M = D*(D+3)//2 # 定数項は回帰変数としてはカウントしない
print('M = %d'%M)

# dataset
X_train = np.zeros((N, M))
i = 0
for d in range(1, D+1):
    for k in range(d+1):
        X_train[:, i] = X1**(d-k) * X2**k
        i += 1

        
X_train[:, 0] = X1
X_train[:, 1] = X2
X_train[:, 2] = X1*X1
X_train[:, 3] = X1*X2
X_train[:, 4] = X2*X2

reg = linear_model.LinearRegression()
reg.fit(X_train, Y)

B_pred = np.zeros(M+1)
B_pred[0] = reg.intercept_
B_pred[1:M+1] = reg.coef_
B_pred

i = 0
Y_pred = np.ones(Y.size) * B_pred[0]
for d in range(1, D+1):
    for k in range(d+1):
        i += 1
        Y_pred += B_pred[i] * X1**(d-k) * X2**k


In [ ]:
def plot_X1X2Y_and_regressionsurface(azimuth=30, elevation=30):
    fig = plt.figure()
    ax = Axes3D(fig)
    ax.view_init(azim=azimuth, elev=elevation)    
    ax.set_xlabel("X1")
    ax.set_ylabel("X2")
    ax.set_zlabel("Y")

    ax.set_zlim(-1, 9)
    ax.scatter(X1, X2, Y, s=40)

    X1mesh = np.arange(np.min(X1), np.max(X1), 0.3)
    X2mesh = np.arange(np.min(X2), np.max(X2), 0.3)

    X1mesh, X2mesh = np.meshgrid(X1mesh, X2mesh)
    Ymesh = B0 + B1*X1mesh + B2*X2mesh + B3*X1mesh**2 + B4*X1mesh*X2mesh + B5*X2mesh**2
    ax.plot_wireframe(X1mesh, X2mesh, Ymesh, linewidth=3, color='tab:orange')
    ax.set_title('MSE = %f'%(np.sum((Y-Y_pred)**2)/Y.size), size=20)

In [ ]:
interact(plot_X1X2Y_and_regressionsurface, azimuth=(0, 360, 1), elevation=(0, 90, 1))

汎化性能のチェックや，最大全次数を変えたときの振る舞いも 3-1 や 3-2 を参考にして計算してみよう．

# 演習問題 3-3

第3回その1で Iris データセットに対して，あらゆる変数の組み合わせについて線形回帰モデルを試した．

では，目的変数と説明変数のあらゆる組み合わせを取り替えて多変数多項式回帰モデルを試し，多項式次数とフィッティング精度に傾向があるか確かめよ．上でやったように多項式次数を十分高く取ると過学習してしまうので，汎化誤差が小さくなるように多項式次数を調整し，学習誤差と汎化誤差の比較も行うこと．

（植物学的に意味があるかはさておき）一番フィッティングの合う多項式回帰モデルは何か．その理由も考えよ．

<h3><div style="text-align: right;">以上</div></h3>